# Conditional replacement of NaN Values in a Pandas Dataframe
This notebook demonstrates how to conditionally fill NaN values in a `precipitation` column of a weather DataFrame, based on values in a `weather` column.

In [32]:
import pandas as pd
import numpy as np

# Example DataFrame
data = {
    'weather': ['rain', 'sun', 'snow', 'drizzle', 'fog', 'rain', 'snow'],
    'precipitation': [5.2, np.nan, 4.1, np.nan, np.nan, np.nan, 3.5]
}
df = pd.DataFrame(data)
df

,weather,precipitation
0,rain,5.2
1,sun,NaN
2,snow,4.1
3,drizzle,NaN
4,fog,NaN
5,rain,NaN
6,snow,3.5


In [33]:
# Define wet weather types
wet_weather = ['rain', 'snow', 'drizzle']

# Calculate mean precipitation 
mean_precip= df.precipitation.mean()

## Method 1: Using `loc` for Conditional Assignment

This is the most straightforward solution.

In [34]:
# Fill NaNs in 'precipitation' where weather is wet
df.loc[df['weather'].isin(wet_weather) & df['precipitation'].isna(), 'precipitation'] = mean_precip 
#loc: [row = (weather is wet and precipitation is NaN), column = precipitation] -> assign the 'mean_precip' value where the condition of the .loc() method are met

df

,weather,precipitation
0,rain,5.200000
1,sun,NaN
2,snow,4.100000
3,drizzle,4.266667
4,fog,NaN
5,rain,4.266667
6,snow,3.500000


## Method 3: Using `.mask()` for Conditional Filling

It is possible to use `.fillna()` — but it’s a bit trickier because `.fillna()` works on a column or Series as a whole, and it doesn't support conditional filling out of the box. However, you can combine it cleverly with `.where()` or `.mask()` to mimic the same behavior.

Here’s a clean way to do it using `.fillna()` with `.mask()`:

`DataFrame.mask(cond, other=<no_default>, *, inplace=False, axis=None, level=None)`

This function replaces values where the __condition is True__

In [35]:
# Recreate the original DataFrame for a fresh example
df = pd.DataFrame(data)

# Fill NaNs conditionally using mask
df['precipitation'] = df['precipitation'].mask(
    df['precipitation'].isna() & df['weather'].isin(wet_weather),
    mean_precip
)
df

,weather,precipitation
0,rain,5.200000
1,sun,NaN
2,snow,4.100000
3,drizzle,4.266667
4,fog,NaN
5,rain,4.266667
6,snow,3.500000


## Method 2: Using `.fillna()` and `.where()` for Conditional Filling

`DataFrame.where(cond, other=nan, *, inplace=False, axis=None, level=None)`

This function replaces values where the __condition is False__

In [36]:

# Recreate the original DataFrame for a fresh example
df = pd.DataFrame(data)

# Fill NaN values in 'precipitation' ONLY for rows with wet weather
df['precipitation'] = df['precipitation'].where(
    ~df['weather'].isin(wet_weather), # Condition: True = weather is NOT wet, False = weather is wet, as the '~' sign (=bitwise NOT operator) inverts True/False
    df['precipitation'].fillna(mean_precip) # Other: Fill the ones where condition is False with 'mean_precip'
)
df

,weather,precipitation
0,rain,5.200000
1,sun,NaN
2,snow,4.100000
3,drizzle,4.266667
4,fog,NaN
5,rain,4.266667
6,snow,3.500000
